In [37]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Image, display
from urllib.parse import urljoin

section_dict = {'최신뉴스':100,'정치':200,'경제':300,'사회':400,'생활/문화':500,'세계':600,'IT/과학':700}


def print_news(section):
    sid = section_dict[section] 
    url = f'https://news.nate.com/recent?mid=n0{sid}'
    print(f'{url} {section} 뉴스')

    req_header = {
        'referer': url,
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }

    res = requests.get(url,headers = req_header)


    if res.ok:
        html = res.text
        soup = BeautifulSoup(html, "html.parser")

        a_tags = soup.select("div.mlt01 a[href*='view'].lt1")

        for a_tag in a_tags:
            if a_tag.text:
                link = a_tag['href']	
                title = a_tag.text.strip()
                img_tag = a_tag.select_one('span.ib img')
                if img_tag:
                    img_url = urljoin(url,img_tag['src'])
                    display(Image(url=img_url))                
                print(title)
                print(f'https:{link}\n')

                

In [38]:
print_news('정치')

https://news.nate.com/recent?mid=n0200 정치 뉴스


민주 "특별재난지역 추가 선포 예정…물가 안정 대책 마련"
																	더불어민주당은 전국적인 폭우 피해와 관련해 중앙재난안전대책본부의 조사가 마무리되면 정부가 특별재난지역을 추가로 선포할 예정이라고 밝혔습니다. 김병기 당 대표 직...
https://news.nate.com/view/20250724n00741?mid=n0200



[투데이 窓]의료 인공지능 '1대 강국'으로 가는 길
																	최윤섭 디지털헬스케어파트너스 대표 필자는 지난 칼럼에서 이번 정부가 천명한 'AI(인공지능) 3대 강국'이라는 야심 찬 목표를 의료AI에서 시작하자는 화두를 던졌다. 한...
https://news.nate.com/view/20250724n00729?mid=n0200



[MT시평]트럼프 칼날, 한국 경제 '넥스트'는?
																	장상식 한국무역협회 국제무역통상연구원장 최근 영국, 베트남, 인도네시아가 미국과 협상을 통해 일부 관세율 인하를 얻어냈지만 한국을 포함한 일본, 유럽연합(EU), 멕시...
https://news.nate.com/view/20250724n00728?mid=n0200



장동혁, 국민의힘 당 대표 출사표…"이기는 정당 만들 것"
																	국민의힘 장동혁 의원이 당 대표 선거 출마를 공식화하며, 내부 총질 대신, 국민의힘을 하나로 모아 이기는 정당을 만들겠다고 강조했습니다. 장 의원은 어제(23일) 국회 ...
https://news.nate.com/view/20250724n00725?mid=n0200



...

'갑질' 강선우 결국 사퇴
																	‘보좌진 갑질’ 논란에 휩싸였던 강선우 여성가족부 장관 후보자가 23일 자진 사퇴했다. 이재명 정부 들어 이진숙 교육부 장관 후보자에 이어 두 번째 장관 후보자 낙마로...
https://news.nate.com/view/20250724n00455?mid=n0200



새 정부 '통합 돌봄' 추진에…요양 병원들 줄폐업 위기
																	“한국 요양 병원이 줄폐업 위기에 처했는데, 일본 요양 병원에선 이런 상황이 없었나요?”이달 초 일본 지바현 가시와시(柏市)의 ‘시 포괄 케어(돌봄) 센터’. 3박 4일 ...
https://news.nate.com/view/20250724n00446?mid=n0200



[단독] 네이버 자회사 취직 '영주권'…야 '아빠 찬스' 의혹
																	<앵커>

네이버 대표를 지낸 최휘영 문화체육관광부 장관 후보자의 딸이 대학 졸업 이후 네이버가 미국에 설립했던 자회사에 취직하고 영주권까지 딴 사실이...
https://news.nate.com/view/20250724n00424?mid=n0200



강선우 "성찰하며 살아가겠다"…지명 30일 만에 사퇴
																	<앵커>

보좌진 갑질 의혹에 휩싸였던 강선우 여성가족부 장관 후보자가 지명된 지 30일 만인 어제(23일) 스스로 물러났습니다. 강 후보자는 성찰하며 살아...
https://news.nate.com/view/20250724n00402?mid=n0200

윤 거부권에 막혔던 '농업2법' 통과
																	농업 재해 범위에 ‘이상 고온’과 ‘지진’을 추가한 농·어업 재해 대책법 개정안과 자연재해로 인한 농·어업인의 손해는 보험료 할증에서 제외하는 농·어업 재해 보험법 개정안이 23일 국회 본회의를 통과했다....
https://news.nate.com/view/20250724n00390?mid=n0200



야당, 혁신은 손놓고…"윤희숙 불렀다" "안 불렀다" 촌극
																	국민의힘 송언석 비상대책위원장 겸 원내대표(왼쪽)와 윤희숙 혁신위원장이 23일 국회 본회의 직후 열린 의원총회에서 만나 악수하고 있다. [연합뉴스] 국민의힘의 쇄신이 ...
https://news.nate.com/view/20250724n00373?mid=n0200

